<a href="https://colab.research.google.com/github/hdilpazir/SmartParking/blob/main/SmartParkingSpotsv1%5Bdone%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code in pycharm is causing overheat hence I have to translate it in Colab

However, for marking the spots we still will need pychram since mouse click are more effective in that environment.

The polycheck idea is from [link](https://pysource.com/2021/10/26/speed-detection-from-cctv-with-opencv-and-deep-learning/)

In [ ]:
# other files
!git clone https://github.com/hdilpazir/smartParkingRelatedFiles.git
%cd /content/smartParkingRelatedFiles

# yolov4 weight file
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
# yolov4-tiny weight file
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

import cv2
import pickle
import numpy as np
from google.colab.patches import cv2_imshow

this is the python code save it in PyCharam as main (after doing the necceasry changes)

In [ ]:
from object_detection import ObjectDetection
# Initialize Object Detection (if using without GPU use Yolo-tiny)
od = ObjectDetection()

cap = cv2.VideoCapture('Khare_testvideo_Cropped.mp4')

# to use this later in each frame for
width_img = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height_img = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) - 10     # OpenCV v2.x used "CV_CAP_PROP_FPS"
size = (width_img, height_img)

# the object to make video in colab
vidstr = cv2.VideoWriter('processed.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         fps, size)

# YOU NEED TO LOAD THE CAR POSITION SPOTS THAT ARE GENERATED USING OTHER PYTHON CODE E.G: markingRectangles.py
with open('carParkPosPoly', 'rb') as f:
    poslist = pickle.load(f)

#------------------------------------------
# to make a list of area for all the four click in one entity of list
polygonlist: list = []
templist: list = []
for pos in poslist:
    if len(templist) <= 3:
        templist.append(pos)
    else:
        polygonlist.append(templist)
        templist: list = []
        templist.append(pos)
# as for last loop else condition
polygonlist.append(templist)
#------------------------------------------

count = 0
while True:
  # to display frames
  success, img = cap.read()

  if success:
    count += 3
    print(count) 
    #count += 5  # i.e. at 30 fps, this advances one second
    cap.set(cv2.CAP_PROP_POS_FRAMES, count)
  else:
    cap.release()
    break

  # Point current frame
  center_points_cur_frame = []

  # Detect objects on frame
  (class_ids, scores, boxes) = od.detect(img)
  for box in boxes:
    (x, y, w, h) = box
    cx = int((x + x + w) / 2)
    cy = int((y + y + h) / 2)
    center_points_cur_frame.append((cx, cy))
    cv2.circle(img, [cx, cy], 2, (255, 0, 255), 1)
    center_points_cur_frame.append([cx, cy])

    
    # stupid logic it draws multiple times:@

    #for loop to fill ones later detect those ones if there is car
    # draw polygon if there is a car
  for area in polygonlist:
    flag = 0
    
    for cxcy in center_points_cur_frame:
      result = cv2.pointPolygonTest(np.array(area, np.int32), cxcy, False)
      if result >= 0:
        flag = 1      
    
    if flag == 1:  
      cv2.polylines(img, np.int32([area]), True, (255, 0, 255), 1)
    else:
      cv2.polylines(img, np.int32([area]), True, (1, 255, 1), 1)

  #cv2_imshow(img)
  
  # the object to write the video in colab
  vidstr.write(img)

#  Just for testing the cv2.pointPolygonTest function

In [ ]:
# checking the function
area = [[354, 418], [392, 477], [506, 477], [451, 404]]
cx = 412
cy = 226
#area = [[0, 4],[0, 0],[4, 0],[4, 4]]
#cx = 4
#cy = 4
result = cv2.pointPolygonTest(np.array(area, np.int32), (int(cx), int(cy)), False)      
print(result)

#  Display video in COLAB 

In [13]:
# define helper function to display videos
import io 
from IPython.display import HTML
from base64 import b64encode
################### Class to be called in the code below
def show_video(file_name, width=640):
  # show resulting deepsort video
  mp4 = open(file_name,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width="{0}" controls>
        <source src="{1}" type="video/mp4">
  </video>
  """.format(width, data_url))
################### Class to be called in the code below

# convert resulting video from avi to mp4 file format
import os
path_video = os.path.join("/content/smartParkingRelatedFiles/","processed.avi")
!ffmpeg -y -loglevel panic -i processed.avi processed.mp4

In [17]:
show_video('/content/smartParkingRelatedFiles/processed.mp4', width=820)